In [ ]:
# The project has two pipelines: Upsampled Lidar and RGB Camera Images
# Camera model architecture

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

# ===========================================
# Required for memory usage in some RTX 2k,3k series

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Memory settings
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

# Fusion: weighting
class c_fusion_wt (layers.Layer):
    def __init__(self,**kwargs):
        super(c_fusion_wt, self).__init__(**kwargs)
    
    def build (self, input_shape):
        self.w=self.add_weight(shape=(1,), initializer="zero", trainable=True,name='weight') 
        
    def call(self, layer_1):
        return layer_1*self.w 

    def get_config(self):
        config = super(c_fusion_wt, self).get_config()
        return config

In [ ]:
#============================================
img_size = (384,1248)
ldr_size= (384,1248) # camera RGB images
class_num = 3 # model output channels


def get_camera_model(img_size, num_classes):
    inputs_img = keras.Input(shape=img_size + (3,), name='rgb')
    
    # Encoder
    #B1
    x = layers.ZeroPadding2D(padding=1, name='Block1_rgb_zp')(inputs_img)
    x = layers.Conv2D(32, 4, strides=2, activation='elu', name='Block1_rgb_conv')(x)
    # Add more encoder blocks as needed
    #B2: Block 2
    x=layers.ZeroPadding2D(padding=1,name='Block2_rgb_zp')(x)
    x=layers.Conv2D(32,3,strides=1,activation='elu',name='Block2_rgb_conv')(x) 

    #B3: Block 3
    x=layers.ZeroPadding2D(padding=1,name='Block3_rgb_zp')(x)
    x=layers.Conv2D(64,4,strides=2,activation='elu', name='Block3_rgb_conv')(x)  
    
    #B4: Block 4
    x=layers.ZeroPadding2D(padding=1, name='Block4_rgb_zp')(x)
    x=layers.Conv2D(64,3,strides=1,activation='elu', name='Block4_rgb_conv')(x) 

    #B5: Block 5
    x=layers.ZeroPadding2D(padding=1,name='Block5_rgb_zp')(x)
    x=layers.Conv2D(128,4,strides=2,activation='elu', name='Block5_rgb_conv')(x) 
    
    #B5: Block 51
    x=layers.ZeroPadding2D(padding=1, name='Block51_rgb_zp')(x)
    x=layers.Conv2D(128,3,strides=1,activation='elu', name='Block52_rgb_conv')(x) 

    #B5: Block 52
    x=layers.ZeroPadding2D(padding=1,name='Block552_rgb_zp')(x)
    x=layers.Conv2D(256,4,strides=2,activation='elu', name='Block552_rgb_conv')(x) 


    # Context module
    #B6: Block 6
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block6_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block6_rgb_dp')(x)

    #B7: Block 7
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block7_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block7_rgb_dp')(x)

    #B8: Block 8
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(1,2),activation='elu', name='Block8_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block8_rgb_dp')(x)

    #B9: Block 9
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(2,4),activation='elu', name='Block9_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block9_rgb_dp')(x)

    #B10: Block 10
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(4,8),activation='elu', name='Block10_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block10_rgb_dp')(x)

    #B11: Block 11
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(8,16),activation='elu', name='Block11_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block11_rgb_dp')(x)

    #B12: Block 12
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(16,32),activation='elu', name='Block12_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block12_rgb_dp')(x)

    #B13: Block 13
    x=layers.Conv2D(256,3,padding="same",dilation_rate=(1,1),activation='elu', name='Block13_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block13_rgb_dp')(x)

    #B14: Block 14
    x=layers.Conv2D(256,1,padding="same",activation='elu', name='Block14_rgb_conv')(x)
    x=layers.Dropout(0.25, name='Block14_rgb_dp')(x)

    # Decoder
    #B15: Block 15
    x=layers.Conv2DTranspose(128,4,strides=2,activation='elu', padding='same', name='Block15_rgb_convtp')(x)

    #B16: Block 16
    x=layers.ZeroPadding2D(padding=1,name='Block16_rgb_zp')(x)
    x=layers.Conv2D(128,3,strides=1,activation='elu', name='Block16_rgb_conv')(x)
    
    #B17: Block 162
    x=layers.Conv2DTranspose(64,4,strides=2,activation='elu', padding='same', name='Block162_rgb_convtp')(x)

    #B18: Block 163
    x=layers.ZeroPadding2D(padding=1, name='Block163_rgb_zp')(x)
    x=layers.Conv2D(64,3,strides=1,activation='elu', name='Block163_rgb_conv')(x)

    #B17: Block 17
    x=layers.Conv2DTranspose(32,4,strides=2,activation='elu', padding='same', name='Block17_rgb_convtp')(x)

    #B18: Block 18
    x=layers.ZeroPadding2D(padding=1, name='Block18_rgb_zp')(x)
    x=layers.Conv2D(32,3,strides=1,activation='elu', name='Block18_rgb_conv')(x)

    #B19: Block 19
    x=layers.Conv2DTranspose(8,4,strides=2, activation='elu', padding='same', name='Block19_rgb_convtp')(x)

    #B20: Block 20
    x=layers.ZeroPadding2D(padding=1, name='Block20_rgb_zp')(x)
    x=layers.Conv2D(class_num,3,strides=1, name='Block20_rgb_conv')(x) 

    # Output layer
    output = layers.Conv2D(num_classes, 3, activation='softmax', padding='same', name='rgb_output')(x)

    # Define the camera model
    camera_model = keras.Model(inputs=inputs_img, outputs=output, name='camera_model')
    return camera_model

camera_model = get_camera_model(img_size, class_num)
camera_model.save('camera_model')